<a href="https://colab.research.google.com/github/gabrielbribeiroo/OperationalResearch_UFPB/blob/main/transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo geral para o problema do transporte

## Instalação e importação do pacote mip

In [ ]:
!pip install mip

from mip import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 23.0 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.




## Leitura dos dados


In [ ]:
def readInstance(filePath):
    f = open(filePath, "r")

    l = f.readline()
    nb_industries, nb_cities = int(l.split()[0]), int(l.split()[1])

    costs = []
    for i in range(nb_industries):
        l = f.readline()
        costs.append([float(c) for c in l.split()])

    l = f.readline()
    capacities = [float(c) for c in l.split()]

    l = f.readline()
    demands = [float(d) for d in l.split()]

    f.close()

    return nb_industries, nb_cities, costs, capacities, demands

## Criação do modelo

In [ ]:
def createModel(nb_industries, nb_cities, costs, capacities, demands):

    model = Model(sense=MINIMIZE, solver_name=CBC)

    x = [[model.add_var(var_type="CONTINUOUS",
                        lb=0.0, name="x_" + str(i) + "_" + str(j)) for j in range(nb_cities)] for i in range(nb_industries)]

    model.objective = xsum(costs[i][j]*x[i][j]
                           for i in range(nb_industries) for j in range(nb_cities))

    for i in range(nb_industries):
        model += xsum(x[i][j] for j in range(nb_cities)
                      ) <= capacities[i], "CAP_" + str(i)

    for j in range(nb_cities):
        model += xsum(x[i][j] for i in range(nb_industries)
                      ) >= demands[j], "DEM_" + str(j)
    return model



## Resolução com Python-MIP

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:

nb_industries, nb_cities, costs, capacities, demands = readInstance("instance.txt")
model = createModel(nb_industries, nb_cities, costs, capacities, demands)

model.write("model.lp")
with open("model.lp") as f: # Lê e exibe conteúdo do arquivo
  print(f.read())

status = model.optimize()

print("Status = ", status)
print("Solution value  = ", model.objective_value)
print("Solution:")
for v in model.vars:
    if v.x > 0.00001:
        print(v.name, " = ", v.x)

x_0_0
\Problem name: 

Minimize
OBJROW: 60 x_0_0 + 220 x_0_1 + 300 x_0_2 + 270 x_0_3 + 450 x_0_4 + 95 x_1_0 + 45 x_1_1 + 200 x_1_2 + 260 x_1_3 + 230 x_1_4
 + 450 x_2_0 + 300 x_2_1 + 250 x_2_2 + 100 x_2_3 + 90 x_2_4
Subject To
CAP_0:  x_0_0 + x_0_1 + x_0_2 + x_0_3 + x_0_4 <= 1100
CAP_1:  x_1_0 + x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 1300
CAP_2:  x_2_0 + x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 1100
DEM_0:  x_0_0 + x_1_0 + x_2_0 >= 400
DEM_1:  x_0_1 + x_1_1 + x_2_1 >= 200
DEM_2:  x_0_2 + x_1_2 + x_2_2 >= 900
DEM_3:  x_0_3 + x_1_3 + x_2_3 >= 1200
DEM_4:  x_0_4 + x_1_4 + x_2_4 >= 750
Bounds
End

Status =  OptimizationStatus.OPTIMAL
Solution value  =  539000.0
Solution:
x_0_0  =  400.0
x_0_3  =  650.0
x_1_1  =  200.0
x_1_2  =  900.0
x_1_4  =  200.0
x_2_3  =  550.0
x_2_4  =  550.0
